In [ ]:
pip install BeautifulSoup4

In [ ]:
pip install python3-wget

In [ ]:
pip install pycurl

In [ ]:
pip install selenium

In [ ]:
pip install html5lib

In [ ]:
pip install webdriver-manager

In [ ]:
pip install uuid

In [ ]:
pip install psycopg2

In [ ]:
# STEP 0: DATABASE TABLES CREATION.

In [ ]:
CREATE EXTENSION pgcrypto;


CREATE TABLE public.metacritic_review_urls(
   id UUID PRIMARY KEY DEFAULT gen_random_uuid(),
   metacritic_href TEXT,
   date_created timestamp with time zone,
   date_updated timestamp with time zone,
   username TEXT
);

CREATE TABLE public.publication_urls(
   id UUID PRIMARY KEY DEFAULT gen_random_uuid(),
   metacritic_id UUID,
   date_created timestamp with time zone,
   date_updated timestamp with time zone,
   username TEXT
);


https://starkandwayne.com/blog/uuid-primary-keys-in-postgresql/
    

In [ ]:
import psycopg2
# method to get a connection to local PostgreSQL database
def get_conn_rds(): 

    try:
        # connect to the postgresql database
        print("Connecting to the PostgreSQL database...")

        connection = psycopg2.connect(user = "postgres",
                                  password = "postgres123",
                                  host = "localhost", 
                                  port = "5432",
                                  database = "postgres")

        return connection

    except (Exception, psycopg2.Error) as error :
        print ("Error while connecting to PostgreSQL", error)
        
conn = get_conn_rds()
print(conn)

In [51]:
import uuid
import datetime

#Insert into table : metacritic_review_urls. 
def insert_to_tbl_metacritic_review_urls(metacritic_href, username):
    try:
        conn = get_conn_rds()
        cur = conn.cursor()
        query_params_list = {'metacritic_href': metacritic_href, 
                             'date_created': datetime.datetime.now(),
                             'date_updated': datetime.datetime.now(),
                             'username': username}
        sql = '''
                INSERT INTO public."metacritic_review_urls"(metacritic_href, date_created, date_updated, username)
                        VALUES (%(metacritic_href)s, %(date_created)s, %(date_updated)s, %(username)s);
                '''
        
        cur.execute(sql, query_params_list)

        conn.commit()
        
        print("Successfully inserted a record")

        cur.close()
        
        print("Closing Connection.")
    
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()  

In [57]:
# STEP 1 - You get urls for all movies on page1 page2 and so on.

In [ ]:
from selenium import webdriver
import uuid
import psycopg2

links_page1 = []
result_urls = []
options = webdriver.ChromeOptions()
options.add_argument('headless')
site_movies_by_year_desc_url = 'https://www.metacritic.com/browse/movies/score/metascore/year/filtered?sort=desc'

# webscraper method to first get all the urls on the above url where I'm filtering "desc" for current year 2020.
# similarly, the above url can be changed to the year 2019. 
# The following method just scrapes all the review urls on page 1. I'll need to do the same for page2 and so on.
def fetchandinsert_movie_releases_by_year_urls():
    browser = webdriver.Chrome("/home/srini/pyprjs/scrpr/chromedriver", options=options)
    browser.get(site_movies_by_year_desc_url)
    css_selector_metacritic_review_links = "div.clamp-metascore"
    divs = browser.find_elements_by_css_selector(css_selector_metacritic_review_links)
    for div in divs:
        href_el = div.find_element_by_css_selector('a').get_attribute('href')
        # inserts the scraped urls on the page into the database table :metacritics_review_urls
        insert_to_metacritic_review_urls(href_el, 'Srini')
        links_page1.append(href_el)
    return links_page1

result_urls = fetchandinsert_movie_releases_by_year_urls()
print(len(result_urls))

In [58]:
# STEP 2 - read the urls from the database and now for each of these urls - we'll need to get publication urls for 
#          critic reviews.

In [ ]:
## 2.a SELECT URLS FROM DATABASE TABLE: metacritic_review_urls

In [ ]:
import psycopg2


def select_from_tbl_metacritic_review_urls():
    
    metacritic_review_urls = dict()
    
    try:
        conn = get_conn_rds()
        cursor = conn.cursor()
        select_query = "select * from public.metacritic_review_urls"
        cursor.execute(select_query)
        print("Selecting rows from the table using cursor.fetchall")
        result_set = cursor.fetchall() 
   
        for row in result_set:
            metacritic_review_urls[row[0]]= row[1]
            
       
    except (Exception, psycopg2.Error) as error :
        print ("Error while fetching data from PostgreSQL", error)

    finally:
    #closing database connection.
        if(conn):
            cursor.close()
            conn.close()
            print("PostgreSQL connection is closed")
    
    return metacritic_review_urls


result_dict = select_from_tbl_metacritic_review_urls()
for key, value in result_dict.items():
    print(f"{key} : {value}")

In [ ]:
## 2.b INSERT INTO TABLE: publication_urls to be used by: fetchandinsert_publication_urls_per_metacritic_url. 
##     Please see 2.c.

In [ ]:
import uuid
import datetime

def insert_to_tbl_publication_urls(metacritic_id, pub_href, username):
    try:
        conn = get_conn_rds()
        cur = conn.cursor()
        query_params_list = {'metacritic_id' : metacritic_id,
                             'pub_href': pub_href, 
                             'date_created': datetime.datetime.now(),
                             'date_updated': datetime.datetime.now(),
                             'username': username}
        sql = '''
                INSERT INTO public."publication_urls"(metacritic_id, pub_href, date_created, date_updated, username)
                        VALUES (%(metacritic_id)s, %(pub_href)s, %(date_created)s, %(date_updated)s, %(username)s);
                '''
        
        cur.execute(sql, query_params_list)

        conn.commit()
        
        print("Successfully inserted a record")

        cur.close()
        
        print("Closing Connection.")
    
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [ ]:
## STPE 3 - Method to fetch the publication urls per metacritic review url.
##     Makes use of methods: 2.a and 2.b.
##     Please note the time.sleep - we'll need to pause a while to not be aggressive.
##     If scraping night time, these could be relaxed, say commenting inner for loop sleep 
##     and changing outer for loop sleep to 60 seconds.

In [ ]:
import time
pub_links =[]
options = webdriver.ChromeOptions()
options.add_argument('headless')
def fetchandinsert_publication_urls_per_metacritic_url():
    
    metacritic_review_urls = select_from_tbl_metacritic_review_urls()  
        
    browser = webdriver.Chrome("/home/srini/pyprjs/scrpr/chromedriver", options=options)
    for metacritic_id, metacritic_href in metacritic_review_urls.items():
            browser.get(metacritic_href)
            webElem = browser.find_elements_by_css_selector("a.read_full")
            for elem in webElem:
                pub_href = elem.get_attribute('href')
                insert_to_tbl_publication_urls(metacritic_id, pub_href, 'Srini')
                pub_links.append(pub_href)
                time.sleep(60)
            time.sleep(120)
        
    
fetchandinsert_publication_urls_per_metacritic_url()        

In [ ]:
# The below two files are the csv exports for review of the data from above two tables: 
# metacritic_review_urls,
# publication_urls 

In [59]:
!ls -lrth *.csv

-rw-r--r-- 1 srini srini  16K Apr 30 22:17 metacritic_review_urls_table_export.csv
-rw-r--r-- 1 srini srini 354K Apr 30 22:18 publication_urls_table_export.csv


In [ ]:
# STEP 4 - tested it out on one url. And this method will change to read publication_url one by one from table:
#          publication_urls and extract <p> tags for paragraphs on each of the publication_url,
#          i.e publication site and the extracted paragraphs will be inserted into the new database table: 
#          pub_review_paragraphs (this still needs to be created and below method refactored for the storing 
#          paragraph text into it.
#          After each pub_url is scraped, we set is_scraped to True - to track completion.
#          After all the pub_urls for a specific metacritic_url are scraped, then is_scraped from 
#          metacritic_review_urls table will be set to : True. 
#          Process goes on until all are completed.

In [ ]:
from bs4 import BeautifulSoup
import requests
def pub_links_scrape_review():
    for link in pub_links[1:2]:
        print(link)
        response = requests.get(link)
        if response.status_code != 200:
            print("Failed to get HTML:", response.status_code, response.reason, link)
        exit()
    html = response.text
    soup = BeautifulSoup(html, "html5lib")
    for ptag in soup.find_all('p'):
        if len(ptag.text) > 100:
            print(ptag.text)
        
pub_links_scrape_review()    